# Tensor
- http://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html

In [1]:
import torch
import numpy as np

In [2]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)
x.size()

torch.Size([5, 3])

### Operation that mutates a tensor in-place is post-fixed with an _

In [3]:
x.t_()


 0.3147  0.3223  0.1950  0.7110  0.3542
 0.5300  0.7438  0.0411  0.8240  0.8776
 0.3090  0.9674  0.0065  0.5039  0.9719
[torch.FloatTensor of size 3x5]

### Indexing

In [4]:
x[:, 1]


 0.3223
 0.7438
 0.9674
[torch.FloatTensor of size 3]

### Torch Tensor <-> Numpy Array

In [5]:
x.numpy()

array([[ 0.31465986,  0.32229367,  0.19503979,  0.71098578,  0.35417312],
       [ 0.52996773,  0.74376523,  0.04108472,  0.82401824,  0.8775906 ],
       [ 0.30898747,  0.96744961,  0.00651642,  0.5039205 ,  0.97189206]], dtype=float32)

In [6]:
torch.from_numpy(np.ones(5))


 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]

### CUDA Tensor

In [7]:
torch.cuda.is_available()

True

In [8]:
x = x.cuda()
y = y.cuda()

In [9]:
x+y


 1.0228  1.2742  1.0701  0.9280  1.3325
 0.5828  0.7967  0.3138  1.6849  1.1624
 1.1694  1.6089  0.6190  0.6335  1.4753
[torch.cuda.FloatTensor of size 3x5 (GPU 0)]

---
# Autograd
- http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

### Variable
- Access raw data through ```.data```, gradient through ```.grad``` attribute
- ```.backward()``` does back propagation

In [10]:
from torch.autograd import Variable

In [11]:
x = Variable(torch.ones(2, 2), requires_grad=True)

In [12]:
y = x + 2
z = y * y * 3
o = z.mean()

In [13]:
o.backward()

In [14]:
print o.grad    # ?

None


---
# Neural Networks
- http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [15]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [16]:
class SampleNet(nn.Module):
    
    def __init__(self):    # define layers with weights
        super(SampleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5, 5))    # 1 input channel, 6 output channel, 5*5 kernel
        self.conv2 = nn.Conv2d(6, 16, (5, 5))
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_features(x))    # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_features(self, x):
        size = x.size()[1:]    # except batch dimension
        n = 1
        for s in size:
            n *= s
        return n        

In [17]:
net = SampleNet()
print net

SampleNet (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [18]:
# Forward
x = Variable(torch.randn(1, 1, 32, 32))    # NCHW
# Input:  1, 32, 32
# Conv1:  6, 28, 28
# Pool1:  6, 14, 14
# Conv2: 16, 10, 10
# Pool2: 16,  5,  5
# FC1:   400 -> 120
# FC2:   120 ->  84
# FC3:    84 ->  10
y_h = net(x)
print y_h

Variable containing:
 0.0173  0.0523 -0.0614 -0.0319  0.0518  0.0708  0.1461  0.0325 -0.0562 -0.0755
[torch.FloatTensor of size 1x10]



In [19]:
# Backward
optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer.zero_grad()    # flush gradient buffer
y = Variable(torch.arange(1, 11))    # dummy
criterion = nn.MSELoss()
loss = criterion(y_h, y)
print loss
loss.backward()    # calc gradients
optimizer.step()    # update weight

Variable containing:
 38.4015
[torch.FloatTensor of size 1]

